In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MaxAbsScaler
from sklearn import metrics
import seaborn as sns

In [2]:
df=pd.read_csv("data/ModelData.csv")
df

SMILES  GPCR_act        MR  \
0      [H]c1nc(C([H])(C#N)c2nc3c([H])c([H])c([H])c([H...  0.251053   79.0614   
1      [H]c1c([H])c(-n2c(=O)n3n(c2=O)C([H])([H])C([H]...  0.129006   63.4810   
2      [H]O[C@]1(c2c([H])oc([H])c2[H])OC(=O)C([H])=C2...  0.798935  125.4138   
3      [H]C1([H])C(=O)N2C([H])([H])[C@]3([H])C([H])([...  0.034461   70.1610   
4      [H]OC1([H])C([H])(c2c([H])nn([H])c2C(=O)N([H])...  0.064913   50.9694   
...                                                  ...       ...       ...   
88873  [H]OC([H])([H])C1=C([H])C([H])([H])[C@]2([H])C...  0.925964   88.2856   
88874  [H]OC(=O)C([H])([H])N([H])C(=O)C1(N([H])C(=O)C...  0.113655   59.0416   
88875  [H]OC(=O)c1c([H])n(O[H])c([H])c([H])/c1=N\c1c(...  0.271177   60.2408   
88876  [H]c1c([H])c([H])c(S(=O)(=O)On2c(=O)c3sc(C([H]...  0.250121   79.4455   
88877  [H]c1c([H])c(S(=O)(=O)OC2([H])C(C([H])([H])[H]...  0.049145   75.8068   

       ATOM       MW    ALOGP  HBA  HBD     PSA  ROTB  AROM  ALERTS       QED  \
0        22  293.286  2.53198  5.0  0.0   97.64   3.0   3.0     2.0  0.545599   
1        18  249.245  0.73360  2.0  0.0   48.93   1.0   2.0     0.0  0.747875   
2        35  482.573  4.45350  7.0  1.0  103.04   2.0   1.0     1.0  0.621189   
3        18  248.370  1.87170  2.0  0.0   23.55   0.0   0.0     0.0  0.653212   
4        15  211.221 -0.42310  4.0  3.0  101.23   2.0   1.0     0.0  0.617763   
...     ...      ...      ...  ...  ...     ...   ...   ...     ...       ...   
88873    22  304.474  3.91860  2.0  2.0   40.46   1.0   0.0     1.0  0.720902   
88874    17  243.263 -1.42510  5.0  4.0  121.52   5.0   0.0     0.0  0.471624   
88875    17  230.223  1.65600  3.0  2.0   74.82   2.0   2.0     1.0  0.770170   
88876    22  339.354  0.27222  6.0  1.0  111.12   3.0   3.0     0.0  0.733170   
88877    20  296.388  2.70402  4.0  0.0   60.44   3.0   1.0     1.0  0.804640   

       Lipinski_Drug-like  Lipinski_Not Drug-like  Veber_Drug-like  \
0                       1                       0                1   
1                       1                       0                1   
2                       1                       0                1   
3                       1                       0                1   
4                       1                       0                1   
...                   ...                     ...              ...   
88873                   1                       0                1   
88874                   1                       0                1   
88875                   1                       0                1   
88876                   1                       0                1   
88877                   1                       0                1   

       Veber_Not Drug-like  Ghose_Drug-like  Ghose_Not Drug-like  
0                        0                1                    0  
1                        0                0                    1  
2                        0                0                    1  
3                        0                0                    1  
4                        0                0                    1  
...                    ...              ...                  ...  
88873                    0                1                    0  
88874                    0                0                    1  
88875                    0                0                    1  
88876                    0                1                    0  
88877                    0                0                    1  

[88878 rows x 19 columns]

# Random Forest
Built to predict the actual value of GPCR_act; includes dummy variables for drug-likeness

In [3]:
labels=np.array(df['GPCR_act'])

df_features=df.drop(['GPCR_act','SMILES'],axis=1)

features_list=list(df.columns)
features_list.remove('GPCR_act')
features_list.remove('SMILES')

features_arr=np.array(df_features)

In [4]:
features_arr

array([[ 79.0614,  22.    , 293.286 , ...,   0.    ,   1.    ,   0.    ],
       [ 63.481 ,  18.    , 249.245 , ...,   0.    ,   0.    ,   1.    ],
       [125.4138,  35.    , 482.573 , ...,   0.    ,   0.    ,   1.    ],
       ...,
       [ 60.2408,  17.    , 230.223 , ...,   0.    ,   0.    ,   1.    ],
       [ 79.4455,  22.    , 339.354 , ...,   0.    ,   1.    ,   0.    ],
       [ 75.8068,  20.    , 296.388 , ...,   0.    ,   0.    ,   1.    ]])

In [5]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features_arr, labels, test_size = 0.25, random_state = 42)

In [6]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (66658, 17)
Training Labels Shape: (66658,)
Testing Features Shape: (22220, 17)
Testing Labels Shape: (22220,)


In [7]:
# The baseline predictions are the historical averages
baseline_preds = labels.mean()
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  0.28


In [8]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [9]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 0.23


In [10]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: -432.04 %.


In [32]:
predictions

array([0.38297387, 0.63318503, 0.27238432, ..., 0.63401077, 0.31465822,
       0.32928206])

In [38]:
print("Model accuracy:",rf.score(test_features,test_labels) * 100,"%")

Model accuracy: 22.01916438332513 %


In [35]:
test_features

array([[ 74.3426,  23.    , 337.254 , ...,   0.    ,   1.    ,   0.    ],
       [115.0865,  31.    , 433.465 , ...,   0.    ,   1.    ,   0.    ],
       [ 66.5091,  16.    , 227.292 , ...,   0.    ,   0.    ,   1.    ],
       ...,
       [151.9268,  39.    , 518.577 , ...,   0.    ,   0.    ,   1.    ],
       [ 58.9787,  14.    , 226.688 , ...,   0.    ,   0.    ,   1.    ],
       [ 59.3964,  14.    , 217.696 , ...,   0.    ,   0.    ,   1.    ]])

In [11]:
test_labels

array([0.79651839, 0.68441498, 0.05477732, ..., 0.00290476, 0.03634668,
       0.1085296 ])

In [12]:
errors

array([0.41354452, 0.05122996, 0.217607  , ..., 0.63110602, 0.27831154,
       0.22075247])

In [13]:
mape

array([5.19190171e+01, 7.48521835e+00, 3.97257501e+02, ...,
       2.17266501e+04, 7.65713762e+02, 2.03403008e+02])

In [14]:
np.mean(mape)

532.0427660101176

In [15]:
errors/test_labels

array([5.19190171e-01, 7.48521835e-02, 3.97257501e+00, ...,
       2.17266501e+02, 7.65713762e+00, 2.03403008e+00])

In [16]:
errors[0]/test_labels[0]

0.5191901707064626

In [30]:
# # Import tools needed for visualization
# from sklearn.tree import export_graphviz
# import pydot
# # Pull out one tree from the forest
# tree = rf.estimators_[5]
# # Import tools needed for visualization
# from sklearn.tree import export_graphviz
# import pydot
# # Pull out one tree from the forest
# tree = rf.estimators_[5]
# # Export the image to a dot file
# export_graphviz(tree, out_file = 'tree.dot', feature_names = features_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
# graph.write_png('tree.png')

KeyboardInterrupt: 

In [28]:
features_list

['MR',
 'ATOM',
 'MW',
 'ALOGP',
 'HBA',
 'HBD',
 'PSA',
 'ROTB',
 'AROM',
 'ALERTS',
 'QED',
 'Lipinski_Drug-like',
 'Lipinski_Not Drug-like',
 'Veber_Drug-like',
 'Veber_Not Drug-like',
 'Ghose_Drug-like',
 'Ghose_Not Drug-like']

## Now RandomForest with Classifiers

In [43]:
threshold=[]
for i in range(0,len(df)):
    if df.loc[i,'GPCR_act']>0.5000000000000:
        threshold.append("Active")
    elif df.loc[i,'GPCR_act']<0.5000000000000:
        threshold.append("Inactive")
    else: threshold.append(None)
       

['Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Active',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Active',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Inactive',
 'Active',
 'Active

In [46]:
df_class=df
df_class['GPCR_act']=threshold

In [47]:
df_class

SMILES  GPCR_act        MR  \
0      [H]c1nc(C([H])(C#N)c2nc3c([H])c([H])c([H])c([H...  Inactive   79.0614   
1      [H]c1c([H])c(-n2c(=O)n3n(c2=O)C([H])([H])C([H]...  Inactive   63.4810   
2      [H]O[C@]1(c2c([H])oc([H])c2[H])OC(=O)C([H])=C2...    Active  125.4138   
3      [H]C1([H])C(=O)N2C([H])([H])[C@]3([H])C([H])([...  Inactive   70.1610   
4      [H]OC1([H])C([H])(c2c([H])nn([H])c2C(=O)N([H])...  Inactive   50.9694   
...                                                  ...       ...       ...   
88873  [H]OC([H])([H])C1=C([H])C([H])([H])[C@]2([H])C...    Active   88.2856   
88874  [H]OC(=O)C([H])([H])N([H])C(=O)C1(N([H])C(=O)C...  Inactive   59.0416   
88875  [H]OC(=O)c1c([H])n(O[H])c([H])c([H])/c1=N\c1c(...  Inactive   60.2408   
88876  [H]c1c([H])c([H])c(S(=O)(=O)On2c(=O)c3sc(C([H]...  Inactive   79.4455   
88877  [H]c1c([H])c(S(=O)(=O)OC2([H])C(C([H])([H])[H]...  Inactive   75.8068   

       ATOM       MW    ALOGP  HBA  HBD     PSA  ROTB  AROM  ALERTS       QED  \
0        22  293.286  2.53198  5.0  0.0   97.64   3.0   3.0     2.0  0.545599   
1        18  249.245  0.73360  2.0  0.0   48.93   1.0   2.0     0.0  0.747875   
2        35  482.573  4.45350  7.0  1.0  103.04   2.0   1.0     1.0  0.621189   
3        18  248.370  1.87170  2.0  0.0   23.55   0.0   0.0     0.0  0.653212   
4        15  211.221 -0.42310  4.0  3.0  101.23   2.0   1.0     0.0  0.617763   
...     ...      ...      ...  ...  ...     ...   ...   ...     ...       ...   
88873    22  304.474  3.91860  2.0  2.0   40.46   1.0   0.0     1.0  0.720902   
88874    17  243.263 -1.42510  5.0  4.0  121.52   5.0   0.0     0.0  0.471624   
88875    17  230.223  1.65600  3.0  2.0   74.82   2.0   2.0     1.0  0.770170   
88876    22  339.354  0.27222  6.0  1.0  111.12   3.0   3.0     0.0  0.733170   
88877    20  296.388  2.70402  4.0  0.0   60.44   3.0   1.0     1.0  0.804640   

       Lipinski_Drug-like  Lipinski_Not Drug-like  Veber_Drug-like  \
0                       1                       0                1   
1                       1                       0                1   
2                       1                       0                1   
3                       1                       0                1   
4                       1                       0                1   
...                   ...                     ...              ...   
88873                   1                       0                1   
88874                   1                       0                1   
88875                   1                       0                1   
88876                   1                       0                1   
88877                   1                       0                1   

       Veber_Not Drug-like  Ghose_Drug-like  Ghose_Not Drug-like  
0                        0                1                    0  
1                        0                0                    1  
2                        0                0                    1  
3                        0                0                    1  
4                        0                0                    1  
...                    ...              ...                  ...  
88873                    0                1                    0  
88874                    0                0                    1  
88875                    0                0                    1  
88876                    0                1                    0  
88877                    0                0                    1  

[88878 rows x 19 columns]